<a href="https://colab.research.google.com/github/mahalakshmijinadoss/pooling/blob/main/hasoc21tamindicbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentencepiece]==4.10.0 datasets --quiet


     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 311 kB 48.8 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 895 kB 49.8 MB/s 
     |████████████████████████████████| 67 kB 3.7 MB/s 
     |████████████████████████████████| 3.3 MB 35.3 MB/s 
     |████████████████████████████████| 1.1 MB 38.3 MB/s 
     |████████████████████████████████| 243 kB 46.5 MB/s 
     |████████████████████████████████| 1.1 MB 44.1 MB/s 
     |████████████████████████████████| 133 kB 48.1 MB/s 
     |████████████████████████████████| 144 kB 49.4 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 271 kB 38.3 MB/s 


In [2]:
!python -c  "import transformers;print(transformers.__version__)"


4.10.0


In [3]:
!pip install -qq ipython-autotime

%load_ext autotime

time: 178 µs (started: 2022-02-10 11:55:51 +00:00)


In [4]:
import torch
torch.cuda.empty_cache()


time: 809 ms (started: 2022-02-10 11:55:51 +00:00)


In [5]:
import numpy as np
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

time: 135 ms (started: 2022-02-10 11:55:52 +00:00)


In [6]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

time: 2.71 s (started: 2022-02-10 11:55:52 +00:00)


In [7]:
import pandas as pd
import numpy as np


time: 2 ms (started: 2022-02-10 11:55:55 +00:00)


In [8]:
from transformers import DataCollatorWithPadding


time: 250 ms (started: 2022-02-10 11:55:55 +00:00)


In [9]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

time: 22.5 s (started: 2022-02-10 11:56:14 +00:00)


In [19]:
train= pd.read_csv('/content/tamil_codemix_offensive_train.tsv',sep='\t')

time: 28.3 ms (started: 2022-02-10 11:57:51 +00:00)


In [20]:
train.shape


(3999, 4)

time: 10.2 ms (started: 2022-02-10 11:57:53 +00:00)


In [21]:
train.head() 


,id,text,category,cleaned_text
0,TA_HL101,@Asha Apo neenga atha government ku theriya pa...,NOT,asha apo neenga atha government ku theriya pad...
1,TA_HL102,@Bala sundar ayyo sorry...antha line ah explai...,NOT,bala sundar ayyo sorry.antha line ah explain p...
2,TA_HL105,@kalimuthu ne ena lusa...yaaru edhu panaalum e...,NOT,kalimuthu ne ena lusa.yaaru edhu panaalum en d...
3,TA_HL109,1st baby ku neat ah feed panunga plzz ipdi iru...,NOT,1st baby ku neat ah feed panunga plzz ipdi iru...
4,TA_HL113,"2012 il vazhgiromnu iruku ithula, apdina?",NOT,"2012 il vazhgiromnu iruku ithula, apdina?"


time: 42.5 ms (started: 2022-02-10 11:57:53 +00:00)


In [30]:
train['category'].value_counts()


0    2019
1    1980
Name: category, dtype: int64

time: 12.6 ms (started: 2022-02-10 11:59:03 +00:00)


In [29]:
train.loc[(train.category == 'NOT'), 'category'] = 0
train.loc[(train.category == 'OFF'), 'category'] = 1
train.loc[(train.category == 'not'), 'category'] = 0
train.loc[(train.category == 'OFf'), 'category'] = 1


time: 34.6 ms (started: 2022-02-10 11:59:01 +00:00)


In [31]:
dev= pd.read_csv('/content/tamil_codemix_offensive_dev.tsv',sep='\t')


time: 21.4 ms (started: 2022-02-10 11:59:22 +00:00)


In [32]:
dev.shape

(940, 4)

time: 5.48 ms (started: 2022-02-10 11:59:24 +00:00)


In [33]:
dev

,id,text,category,cleaned_text
0,TA_TW15946,Take it this thevidiya Kandipa indha page admi...,OFF,take it this thevidiya kandipa indha page admi...
1,TA_TW10175,enga veetla itha nadakum Athum oru varushama t...,NOT,enga veetla itha nadakum athum oru varushama t...
2,TA_TW15947,"Indha Sallli Punda, Dummy Pundalam Orama Iruka...",OFF,"indha sallli punda, dummy pundalam orama iruka..."
3,TA_TW15174,Juriya poola tier 1 la umburan tha kulla punda...,OFF,juriya poola tier 1 la umburan tha kulla punda...
4,TA_TW15182,Kullans lam umba therila Loosu kuthi maari umb...,OFF,kullans lam umba therila loosu kuthi maari umb...
...,...,...,...,...
935,TA_TW15928,Woman with a bhindi picture on a logo promotes...,OFF,woman with a bhindi picture on a logo promotes...
936,TA_TW10014,Worst ra Vara vara Namma Society kevalama poit...,OFF,worst ra vara vara namma society kevalama poit...
937,TA_TW10134,Yeallarukum reply pandringa namma #TAG RakidaR...,NOT,yeallarukum reply pandringa namma tag rakidara...
938,TA_TW10418,Yeva yeva valkaiyelam flashlight adicha maari ...,NOT,yeva yeva valkaiyelam flashlight adicha maari ...


time: 21.4 ms (started: 2022-02-10 11:59:26 +00:00)


In [34]:
dev['category'].value_counts()


OFF    475
NOT    465
Name: category, dtype: int64

time: 23.2 ms (started: 2022-02-10 11:59:48 +00:00)


In [36]:
dev.loc[(dev.category == 'NOT'), 'category'] = 0
dev.loc[(dev.category == 'OFF'), 'category'] = 1


time: 4.65 ms (started: 2022-02-10 12:00:13 +00:00)


In [37]:
import re

time: 1.37 ms (started: 2022-02-10 12:00:17 +00:00)


In [38]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

time: 5.19 ms (started: 2022-02-10 12:00:18 +00:00)


In [39]:
X_train=train.text.values
X_dev=dev.text.values
y_train=train.category.values
y_dev=dev.category.values

time: 11.8 ms (started: 2022-02-10 12:00:38 +00:00)


In [40]:
train_texts= np.array([preprocess(text) for text in X_train])
dev_texts= np.array([preprocess(text) for text in X_dev])
num_labels=2


time: 124 ms (started: 2022-02-10 12:00:40 +00:00)


In [41]:
train_texts=train_texts.tolist()
dev_texts=dev_texts.tolist()

time: 8.79 ms (started: 2022-02-10 12:00:41 +00:00)


In [42]:
train_texts[0:20]

['@user Apo neenga atha government ku theriya paduthaliya',
 '@user sundar ayyo sorry...antha line ah explain pannama vittutu irukan[:drowsy]ok na solran( en appavum indha award function ku vanthurukkaru,neenga en appava paakala pola....en appavukku munnadiye ipdi enna insult panra maathri kevi kettu asinga paduthuringa nu solraaru[:yeah][:yeah] ‘\xa0chiiiii karumam podinnngggg... asingama vaila vanthurum....',
 '@user ne ena lusa...yaaru edhu panaalum en da religion ah ilukreenga...manishana paaru da modhalla...',
 "1st baby ku neat ah feed panunga plzz ipdi irundha epdi sapdum baby udambu full ah agitu plzz don't force tooo by doing this type of method [",
 '2012 il vazhgiromnu iruku ithula, apdina?',
 '30 varusa kadan. 25 age engayo idikuthe',
 'a vanitha veliya po ethuku thirumpi vantha',
 'à®•à¯\x81à®´à®¨à¯\x8dà®¤à¯ˆ ga taste ah saptanum nu ninappanga nenga eappo pathalum paal satham, paruppu, neai, nu thantha avanga saptave pidikkathu nenga mathi mathi tharanum',
 'Aaiiii Jolly Y

time: 24.5 ms (started: 2022-02-10 12:00:43 +00:00)


In [43]:
train_encodings = tokenizer(train_texts,truncation=True,)
dev_encodings = tokenizer(dev_texts,truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


time: 702 ms (started: 2022-02-10 12:00:46 +00:00)


In [44]:
import torch

class TaskDatasetB(torch.utils.data.Dataset):

  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

time: 6.76 ms (started: 2022-02-10 12:00:48 +00:00)


In [45]:
train_dataset = TaskDatasetB(train_encodings,y_train)
dev_dataset = TaskDatasetB(dev_encodings,y_dev)

time: 1.56 ms (started: 2022-02-10 12:00:51 +00:00)


In [46]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


time: 930 µs (started: 2022-02-10 12:00:53 +00:00)


In [47]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'OGBV-gender-indicbert-ta-hasoc21',
    run_name = "twt-indicbert-ta-3epoch",
    evaluation_strategy = 'epoch',
    save_strategy='epoch',
    num_train_epochs = 3,
    load_best_model_at_end = True,
    push_to_hub='OGBV-gender-indicbert-ta-hasoc21'
)

time: 121 ms (started: 2022-02-10 12:01:08 +00:00)


In [48]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def compute_metrics(eval_preds):

  logits,labels = eval_preds
  predictions  = np.argmax(logits,axis=-1)

  acc = accuracy_score(labels,predictions)
  precision = precision_score(labels,predictions,average='weighted')
  recall = recall_score(labels,predictions,average='weighted')
  f1 = f1_score(labels,predictions,average='weighted')
  
  return {
     'acc' : acc,
     'precision' : precision,
     'recall' : recall,
     'f1' : f1
  }

time: 9.5 ms (started: 2022-02-10 12:01:10 +00:00)


In [49]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [50]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,

    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Maha/OGBV-gender-indicbert-ta-hasoc21 into local empty directory.


time: 20.6 s (started: 2022-02-10 12:01:25 +00:00)


In [51]:
trainer.train()


***** Running training *****
  Num examples = 3999
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1500


Epoch,Training Loss,Validation Loss,Acc,Precision,Recall,F1
1,0.697400,0.693731,0.507447,0.565088,0.507447,0.349493
2,0.685300,0.623568,0.675532,0.790695,0.675532,0.641533
3,0.513100,0.482454,0.767021,0.775184,0.767021,0.765576


***** Running Evaluation *****
  Num examples = 940
  Batch size = 8
Saving model checkpoint to OGBV-gender-indicbert-ta-hasoc21/checkpoint-500
Configuration saved in OGBV-gender-indicbert-ta-hasoc21/checkpoint-500/config.json
Model weights saved in OGBV-gender-indicbert-ta-hasoc21/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 940
  Batch size = 8
Saving model checkpoint to OGBV-gender-indicbert-ta-hasoc21/checkpoint-1000
Configuration saved in OGBV-gender-indicbert-ta-hasoc21/checkpoint-1000/config.json
Model weights saved in OGBV-gender-indicbert-ta-hasoc21/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 940
  Batch size = 8
Saving model checkpoint to OGBV-gender-indicbert-ta-hasoc21/checkpoint-1500
Configuration saved in OGBV-gender-indicbert-ta-hasoc21/checkpoint-1500/config.json
Model weights saved in OGBV-gender-indicbert-ta-hasoc21/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share y

TrainOutput(global_step=1500, training_loss=0.6319362996419271, metrics={'train_runtime': 415.821, 'train_samples_per_second': 28.851, 'train_steps_per_second': 3.607, 'total_flos': 43749722785860.0, 'train_loss': 0.6319362996419271, 'epoch': 3.0})

time: 6min 55s (started: 2022-02-10 12:01:50 +00:00)


In [52]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 940
  Batch size = 8


{'epoch': 3.0,
 'eval_acc': 0.7670212765957447,
 'eval_f1': 0.7655762257715589,
 'eval_loss': 0.4824535846710205,
 'eval_precision': 0.7751842778536302,
 'eval_recall': 0.7670212765957447,
 'eval_runtime': 11.2083,
 'eval_samples_per_second': 83.866,
 'eval_steps_per_second': 10.528}

time: 11.2 s (started: 2022-02-10 12:08:52 +00:00)


In [53]:
import torch
torch.cuda.empty_cache()

time: 131 ms (started: 2022-02-10 12:09:06 +00:00)


In [54]:
model_path = 'OGBV-gender-indicbert-ta-hasoc21'


time: 1.2 ms (started: 2022-02-10 12:09:08 +00:00)


In [55]:
model.push_to_hub('OGBV-gender-indicbert-ta-hasoc21')


Configuration saved in OGBV-gender-indicbert-ta-hasoc21/config.json
Model weights saved in OGBV-gender-indicbert-ta-hasoc21/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.37k/128M [00:00<?, ?B/s]

Upload file runs/Feb10_12-01-08_478acb4f4947/events.out.tfevents.1644494510.478acb4f4947.80.0:  64%|######4   …

Upload file runs/Feb10_12-01-08_478acb4f4947/events.out.tfevents.1644494943.478acb4f4947.80.2: 100%|##########…

Upload file runs/Feb10_12-01-08_478acb4f4947/1644494510.8547022/events.out.tfevents.1644494510.478acb4f4947.80…

To https://huggingface.co/Maha/OGBV-gender-indicbert-ta-hasoc21
   0f8b7ab..ca5f9e8  main -> main



'https://huggingface.co/Maha/OGBV-gender-indicbert-ta-hasoc21/commit/ca5f9e8eae2e43a68ac2a3b9651d0db5ece34ba4'

time: 2min 11s (started: 2022-02-10 12:09:11 +00:00)


In [56]:
tokenizer.push_to_hub('OGBV-gender-indicbert-ta-hasoc21')


tokenizer config file saved in OGBV-gender-indicbert-ta-hasoc21/tokenizer_config.json
Special tokens file saved in OGBV-gender-indicbert-ta-hasoc21/special_tokens_map.json


Upload file spiece.model:   0%|          | 3.38k/5.38M [00:00<?, ?B/s]

To https://huggingface.co/Maha/OGBV-gender-indicbert-ta-hasoc21
   ca5f9e8..226c158  main -> main



'https://huggingface.co/Maha/OGBV-gender-indicbert-ta-hasoc21/commit/226c158ba791cdae6a5ca3cbe5dc3b324424ad3b'

time: 13.6 s (started: 2022-02-10 12:11:28 +00:00)
